In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
from torch.nn.utils.rnn import pad_sequence
import numpy as np

In [2]:
class NERset(Dataset):
    def __init__(self,root,mode):
        with open(root, "rb") as f:
            self.data = pickle.load(f)
        self.train_or_test = mode

    def __getitem__(self, index):
        # sample[0]:name
        # sample[1]:text_tag
        # sample[2]:segments_tensor
        # sample[3]:index
        # sample[4]:index_bound
        # sample[5]:answer_able
        # sample[6]:start_label
        # sample[7]:end_label
        name = self.data[index]['file']
        text_tokens_tensors = self.data[index]['text']
        tag_tokens_tensors = self.data[index]['tag']
        r_index = self.data[index]['index']
        r_index_bound = self.data[index]['index_bound']
        if self.train_or_test=='train':
            start_label_ids = self.data[index]['val_start']
            end_label_ids = self.data[index]['val_end']
            answer_able = self.data[index]['able']
        elif self.train_or_test=='test':
            start_label_ids = None
            end_label_ids = None
            answer_able = None
        text_len = text_tokens_tensors.shape[0]
        tag_len = tag_tokens_tensors.shape[0]
        segments_tensor = torch.tensor([0]*text_len +[1]*tag_len)
        text_tag_tensors = torch.cat((text_tokens_tensors,tag_tokens_tensors))
        
        total_len = text_len + tag_len
        # max_tag_len = 16
        # max_text_len = 1003
        if total_len >=512:
            text_tokens_tensors = self.data[index]['text'][0:495]
            
            sep_tensor = torch.tensor([3])
            text_tag_tensors = torch.cat((text_tokens_tensors,sep_tensor))
            
            text_tag_tensors = torch.cat((text_tag_tensors,tag_tokens_tensors))
            text_len = 496
            segments_tensor = torch.tensor([0]*text_len +[1]*tag_len)
            
            
        
        if end_label_ids !=None and end_label_ids > text_len:
            answer_able = 0
            start_label_ids = -1
            end_label_ids = -1
            
        
        return (name,text_tag_tensors,segments_tensor,r_index,r_index_bound,answer_able,start_label_ids,end_label_ids)
        

    def __len__(self):
        return len(self.data)
    def create_mini_batch(self,samples):
        # sample[0]:name
        # sample[1]:text_tag
        # sample[2]:segments_tensor
        # sample[3]:mask_tensor
        # sample[4]:index
        # sample[5]:index_bound
        # sample[6]:answer_able
        # sample[7]:start_label
        # sample[8]:end_label
       
        
        name = [s[0] for s in samples]
        text_tag_tensors = [s[1] for s in samples]
        segments_tensors = [s[2] for s in samples]
        index = [s[3] for s in samples]
        index_bound = [s[4] for s in samples]
        if self.train_or_test=='train':
            answerable = [s[5] for s in samples]
            start_tensors= [s[6] for s in samples]
            end_tensors= [s[7] for s in samples]
            answerable = torch.tensor([i for i in (answerable)])
            start_tensors  = torch.tensor([i for i in (start_tensors)])
            end_tensors = torch.tensor([i for i in (end_tensors)])
        else:
            answerable =None
            start_tensors = None
            end_tensors= None
        
       
        text_tag_tensors = pad_sequence(text_tag_tensors,batch_first=True)
        segments_tensors = pad_sequence(segments_tensors,batch_first=True)
        masks_tensors = torch.zeros(text_tag_tensors.shape,dtype=torch.long)
        masks_tensors = masks_tensors.masked_fill(text_tag_tensors != 0, 1)
        
        
            
        return name,text_tag_tensors,segments_tensors,masks_tensors,index,index_bound,answerable,start_tensors,end_tensors

In [6]:
if __name__ == "__main__":
    
    dataset = NERset(root='textData_new.pkl',mode='train')
    dataloader = DataLoader(dataset, batch_size=6, shuffle=False,collate_fn=dataset.create_mini_batch)
    for i,data in enumerate(dataloader):
        if data[1].shape[1] >500:
            print(data)
            
            
    

        
        

(['300366034.pdf.xlsx', '300366034.pdf.xlsx', '300366034.pdf.xlsx', '300366034.pdf.xlsx', '300366034.pdf.xlsx', '300366034.pdf.xlsx'], tensor([[  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,  25, 253,   3],
        [  2,   1,   1,  ..., 483, 238,   3]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), [[13, 14, 15, 16, 17, 18, 19], [13, 14, 15, 16, 17, 18, 19], [13, 14, 15, 16, 17, 18, 19], [13, 14, 15, 16, 17, 18, 19], [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3

(['300364838.pdf.xlsx', '300364838.pdf.xlsx', '300364838.pdf.xlsx', '300364838.pdf.xlsx', '300364838.pdf.xlsx', '300364838.pdf.xlsx'], tensor([[  2,   1, 475,  ..., 194,   1,   3],
        [  2,   1, 475,  ...,   0,   0,   0],
        [  2,   1, 475,  ...,   0,   0,   0],
        [  2,   1, 475,  ...,   0,   0,   0],
        [  2,   1, 475,  ..., 553,   3,   0],
        [  2,   1, 475,  ..., 107,   3,   0]]), tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 1, 1, 0]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0]]), [[9, 10, 13, 14, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], [9, 10, 13, 14, 16, 18, 19, 20, 21, 22, 23, 24, 25, 2

(['300365616.pdf.xlsx', '300365616.pdf.xlsx', '300365616.pdf.xlsx', '300365616.pdf.xlsx', '300365616.pdf.xlsx', '300365616.pdf.xlsx'], tensor([[   2,  164, 1137,  ...,    0,    0,    0],
        [   2,  164, 1137,  ...,    0,    0,    0],
        [   2,    1,    1,  ...,   25,  253,    3],
        [   2,    1,    1,  ...,  483,  238,    3],
        [   2,    1,    1,  ...,  573,  143,    3],
        [   2,    1,    1,  ...,  143,    3,    0]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0]]), [[2, 3, 4, 5], [2, 3, 4, 5], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 

(['300360248.pdf.xlsx', '300360248.pdf.xlsx', '300360248.pdf.xlsx', '300360248.pdf.xlsx', '300360248.pdf.xlsx', '300360248.pdf.xlsx'], tensor([[  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,  25, 253,   3],
        [  2,   1,   1,  ..., 483, 238,   3]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), [[26, 27, 28, 29, 30, 31, 32], [26, 27, 28, 29, 30, 31, 32], [26, 27, 28, 29, 30, 31, 32], [26, 27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 4

(['300366500.pdf.xlsx', '300366500.pdf.xlsx', '300366500.pdf.xlsx', '300366500.pdf.xlsx', '300366500.pdf.xlsx', '300366500.pdf.xlsx'], tensor([[  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,  25, 253,   3],
        [  2,   1,   1,  ..., 483, 238,   3]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), [[7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [18, 19, 20, 21, 2

(['300359906.pdf.xlsx', '300359906.pdf.xlsx', '300359906.pdf.xlsx', '300359906.pdf.xlsx', '300359906.pdf.xlsx', '300359906.pdf.xlsx'], tensor([[  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ..., 194,   1,   3],
        [  2,   1,   1,  ...,   0,   0,   0]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), [[65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88], [65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,

In [7]:
if __name__ == "__main__":
    
    dataset = NERset(root='textData_test.pkl',mode='test')
    dataloader = DataLoader(dataset, batch_size=6, shuffle=False,collate_fn=dataset.create_mini_batch)
    for i,data in enumerate(dataloader):
        if data[1].shape[1] >500:
            print(data)
            #print(data[1].shape,data[2].shape,data[3].shape)
            #print(data[4])
            #print(data[5])

(['300365639.pdf.xlsx', '300365639.pdf.xlsx', '300365639.pdf.xlsx', '300365639.pdf.xlsx', '300365639.pdf.xlsx', '300365639.pdf.xlsx'], tensor([[  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ..., 194,   1,   3],
        [  2,   1,   1,  ...,   0,   0,   0]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), [[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38], [21, 22, 23, 24, 25, 26, 2

(['300366168.pdf.xlsx', '300366168.pdf.xlsx', '300366168.pdf.xlsx', '300366168.pdf.xlsx', '300366168.pdf.xlsx', '300366168.pdf.xlsx'], tensor([[  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ...,   0,   0,   0],
        [  2,   1,   1,  ..., 194,   1,   3],
        [  2,   1,   1,  ...,   0,   0,   0]]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), [[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 3

In [4]:
print((data[0]))

['300365804.pdf.xlsx', '300365804.pdf.xlsx', '300365804.pdf.xlsx', '300365804.pdf.xlsx', '300365804.pdf.xlsx', '300365804.pdf.xlsx']
